In [1]:
import json
import random
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

import os
import string
from collections import defaultdict
import heapq
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import spacy
import torch
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertModel, BertTokenizer


i:\conda_envs\dspy_dfkg\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
class GraphAligner:
    def __init__(self, graph1_path, graph2_path, model_name='all-MiniLM-L6-v2'):
        self.model = SentenceTransformer(model_name)
        self.matched_pairs = {}
        self.new_id_start_graphA = 10000
        self.new_id_start_graphB = 20000
        self.graph1 = self.rename_ids(self.load_json(graph1_path), graph_type="A")
        self.graph2 = self.rename_ids(self.load_json(graph2_path), graph_type="B")

    def load_json(self, file_path):
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                return json.load(file)
        except FileNotFoundError:
            print(f"Error: File not found - {file_path}")
        except json.JSONDecodeError as e:
            print(f"Error: Failed to decode JSON from file - {file_path}")
            print(f"Error details: {e}")
        return None

    def rename_ids(self, json_obj, graph_type="A"):
        new_id = (
            self.new_id_start_graphA if graph_type == "A" else self.new_id_start_graphB
        )  # we relabel all IDs with larger numbers
        # Create a mapping of old IDs to new IDs
        id_mapping = {}

        for item in json_obj:
            old_id = item['id']
            id_mapping[old_id] = str(new_id)
            new_id += 1

        # Update IDs and references
        for item in json_obj:
            item['id'] = id_mapping[item['id']]

            if 'source_ref' in item:
                item['source_ref'] = id_mapping[item['source_ref']]

            if 'target_ref' in item:
                item['target_ref'] = id_mapping[item['target_ref']]

        # Print the updated data
        return json_obj

    # All nested attributes within one node/edge are flatten
    # a: {b: v1, c: v2} is flatten to a dictionary {a.b: v1, a.c: v2}
    def flatten_json(self,anNode_or_EdgeInSTIX):
        out = {}

        def flatten(x, name=''):
            if isinstance(x, dict):
                for a in x:
                    flatten(x[a], name + a + '.')
            elif isinstance(x, list):
                for i, a in enumerate(x):
                    flatten(a, name + str(i) + '.')
            else:
                out[name[:-1]] = x

        flatten(anNode_or_EdgeInSTIX)
        return out

    # input is a dict
    def node_or_edge_to_text(self,anFlattenNode_or_EdgeInSTIX):
        text = ". ".join(f"{key}: {value}" for key, value in anFlattenNode_or_EdgeInSTIX.items())
        return text

    # input is a graph
    def graph_to_text(self, a_graph):
        a_graph_text={}
        for node_or_edge in a_graph:
            flattened_node_or_edge = self.flatten_json(node_or_edge)
            # We don't need the id and we will reassign a new ID
            original_id = flattened_node_or_edge.pop('id')
            a_graph_text[original_id]=self.node_or_edge_to_text(flattened_node_or_edge)

        return a_graph_text    

    def generate_element_embeddings(self, a_graph):
        graph_texts=self.graph_to_text(a_graph)
        # Generate embeddings by converting object attributes to a single string
        return {key: self.model.encode(text) for key, text in graph_texts.items()}

    def compute_similarity_matrix(self, file="similarity_matrix.txt"):
        embeddings1=np.vstack(list(self.generate_element_embeddings(self.graph1).values()))
        embeddings2=np.vstack(list(self.generate_element_embeddings(self.graph2).values()))

        similarity=cosine_similarity(embeddings1, embeddings2)

        with open(file, "w") as f:
            for row in similarity:
                f.write(" ".join([str(x) for x in row]) + "\n")

        return similarity

    def match_pairs(self, file="similarity_matrix.txt", threshold=0.8):
        if not os.path.exists("similarity_matrix.txt"): # check if the file exists
            self.compute_similarity_matrix() # compute the similarity matrix

        with open(file) as f:
            similarity = [[float(x) for x in line.strip().split()] for line in f] # read the matrix from the file

        self.matched_pairs = {}
        for i, row in enumerate(similarity):
            max_score = max(row)
            if max_score > threshold:
                j = row.index(max_score)
                if (j, i) not in self.matched_pairs and (i, j) not in self.matched_pairs: # check if the pair is already matched or reversed
                    self.matched_pairs[i] = j

    def replace_id(
        self, a_graph, id_name, matched_pairs, shown_old_ids=False, graph_type="A"
    ):
        id_start = (
            self.new_id_start_graphA if graph_type == "A" else self.new_id_start_graphB
        )
        for obj in a_graph:
            # make sure we found ID, source_ref, target_ref
            if id_name in obj.keys():
                key = int(obj[id_name]) - id_start
                if key in matched_pairs.keys():
                    if shown_old_ids:
                        obj["old_"+id_name] = key
                    obj[id_name] = str(matched_pairs[key])

                else:
                    if shown_old_ids:
                        obj["old_" + id_name] = obj[id_name]
                    obj[id_name] = obj[id_name]

    def align_graph(self, a_graph, matched_pairs, shown_old_ids=False, graph_type="A"):
        # Update a graph with new IDs
        self.replace_id(a_graph, "id", matched_pairs, shown_old_ids, graph_type)
        self.replace_id(a_graph, "source_ref", matched_pairs, shown_old_ids, graph_type)
        self.replace_id(a_graph, "target_ref", matched_pairs, shown_old_ids, graph_type)

        return a_graph

    def align_graphs(self, shown_old_ids=False):
        self.match_pairs(file="similarity_matrix.txt", threshold=0.8)

        self.graph1 = self.align_graph(
            self.graph1, self.matched_pairs, shown_old_ids, graph_type="A"
        )
        self.graph2 = self.align_graph(
            self.graph2, self.matched_pairs, shown_old_ids, graph_type="B"
        )

        return self.graph1, self.graph2

In [3]:
# Example usage
graph1_path = 'testcase/node_distance/06_a.json'  # Path to your first graph JSON file
graph2_path = "testcase/node_distance/06_b.json"  # Path to your second graph JSON file

aligner = GraphAligner(graph1_path, graph2_path)
matrix=aligner.compute_similarity_matrix()
matched_pair=aligner.match_pairs()

# print("Aligned Graph 1:", json.dumps(aligned_graph1, indent=2))
# print("Aligned Graph 2:", json.dumps(aligned_graph2, indent=2))

i:\conda_envs\dspy_dfkg\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
aligner.load_json(graph1_path)

[{'id': '1',
  'name': 'Hillary Clinton',
  'description': 'Hillary Diane Rodham Clinton is an American politician',
  'age': 70,
  'birth': 'US'},
 {'id': '2',
  'name': 'Christopher G. Kollmann',
  'description': 'Forensic investigator',
  'age': 48,
  'birth': 'US'},
 {'id': '3', 'source_ref': '1', 'target_ref': '2'}]

In [5]:
aligner.graph1

[{'id': '10000',
  'name': 'Hillary Clinton',
  'description': 'Hillary Diane Rodham Clinton is an American politician',
  'age': 70,
  'birth': 'US'},
 {'id': '10001',
  'name': 'Christopher G. Kollmann',
  'description': 'Forensic investigator',
  'age': 48,
  'birth': 'US'},
 {'id': '10002', 'source_ref': '10000', 'target_ref': '10001'}]

In [6]:
aligner.graph2

[{'id': '20000',
  'name': 'Hillary Clinton',
  'description': 'Hillary Diane Rodham Clinton is an American politician',
  'age': 70,
  'birth': 'US'},
 {'id': '20001',
  'name': 'Elon Musk',
  'description': 'Elon Reeve Musk is a businessman and investor.',
  'age': 52,
  'birth': 'South Africa'},
 {'id': '20002', 'source_ref': '20000', 'target_ref': '20001'}]

In [7]:
print(matrix)

[[1.0000001  0.31793803 0.05827784]
 [0.24655926 0.39193857 0.15183762]
 [0.04843677 0.08228613 0.93008065]]


In [8]:
print(aligner.matched_pairs)

{0: 0, 2: 2}


In [9]:
g1, g2=aligner.align_graphs(shown_old_ids=False)

In [10]:
g1

[{'id': '0',
  'name': 'Hillary Clinton',
  'description': 'Hillary Diane Rodham Clinton is an American politician',
  'age': 70,
  'birth': 'US'},
 {'id': '10001',
  'name': 'Christopher G. Kollmann',
  'description': 'Forensic investigator',
  'age': 48,
  'birth': 'US'},
 {'id': '2', 'source_ref': '0', 'target_ref': '10001'}]

In [11]:
g2

[{'id': '0',
  'name': 'Hillary Clinton',
  'description': 'Hillary Diane Rodham Clinton is an American politician',
  'age': 70,
  'birth': 'US'},
 {'id': '20001',
  'name': 'Elon Musk',
  'description': 'Elon Reeve Musk is a businessman and investor.',
  'age': 52,
  'birth': 'South Africa'},
 {'id': '2', 'source_ref': '0', 'target_ref': '20001'}]